In [1]:
from simul import benchmark

In [2]:
%timeit benchmark()
#result 12.9 s ± 83.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

14.2 s ± 1.54 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
import cProfile
pr = cProfile.Profile()
pr.enable()
benchmark()
pr.disable()
pr.print_stats()

         7023 function calls in 12.381 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   12.381   12.381 <ipython-input-3-58d835d77762>:4(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-58d835d77762>:5(<module>)
        2    0.000    0.000    0.000    0.000 codeop.py:132(__call__)
        2    0.000    0.000    0.000    0.000 hooks.py:142(__call__)
        2    0.000    0.000    0.000    0.000 hooks.py:207(pre_run_code_hook)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:1258(user_global_ns)
        2    0.000    0.000   12.381    6.190 interactiveshell.py:3230(run_code)
        2    0.000    0.000    0.000    0.000 ipstruct.py:125(__getattr__)
     3000    0.003    0.000    0.004    0.000 random.py:371(uniform)
        1    0.000    0.000   12.381   12.381 simul.py:114(benchmark)
        1    0.002    0.002    0.007    0.007 simul.py:115(<listcomp>)
  

In [5]:
%prun benchmark()
#result

#          7007 function calls in 11.932 seconds

#    Ordered by: internal time

#    ncalls  tottime  percall  cumtime  percall filename:lineno(function)
#         1   11.929   11.929   11.929   11.929 simul.py:15(evolve)
#         1    0.002    0.002    0.003    0.003 simul.py:115(<listcomp>)
#      3000    0.001    0.000    0.001    0.000 random.py:371(uniform)
#      1000    0.000    0.000    0.000    0.000 simul.py:8(__init__)
#      3000    0.000    0.000    0.000    0.000 {method 'random' of '_random.Random' objects}
#         1    0.000    0.000   11.932   11.932 <string>:1(<module>)
#         1    0.000    0.000   11.932   11.932 {built-in method builtins.exec}
#         1    0.000    0.000   11.932   11.932 simul.py:114(benchmark)
#         1    0.000    0.000    0.000    0.000 simul.py:13(__init__)
#         1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


In [6]:
%load_ext line_profiler

In [7]:
from simul import ParticleSimulator,benchmark 
#優化前
%lprun -f ParticleSimulator.evolve benchmark()
#result

# Timer unit: 3.95062e-07 s

# Total time: 136.565 s
# File: C:\Users\user\Documents\python\machinelearning\optimize\simul.py
# Function: evolve at line 15

# Line #      Hits         Time  Per Hit   % Time  Line Contents
# ==============================================================
#     15                                               def evolve(self, dt):
#     16         1         13.0     13.0      0.0          timestep = 0.00001
#     17         1         15.0     15.0      0.0          nsteps = int(dt/timestep)
#     18                                                   
#     19     10001      55569.0      5.6      0.0          for i in range(nsteps):
#     20  10010000   31380132.0      3.1      9.1              for p in self.particles:
#     21                                                           #計算方向
#     22  10000000   82864277.0      8.3     24.0                  norm = (p.x**2+p.y**2)**0.5
#     23  10000000   40156089.0      4.0     11.6                  v_x = -p.y/norm
#     24  10000000   34617088.0      3.5     10.0                  v_y = p.x/norm
#     25                                                           
#     26                                                           #計算位移
#     27  10000000   39929310.0      4.0     11.6                  d_x = timestep * p.ang_vel * v_x
#     28  10000000   37301958.0      3.7     10.8                  d_y = timestep * p.ang_vel * v_y
#     29                                                           
#     30  10000000   40280404.0      4.0     11.7                  p.x += d_x
#     31  10000000   39093998.0      3.9     11.3                  p.y += d_y


In [8]:
#優化後
from simul_optimize import ParticleSimulator,benchmark
%lprun -f ParticleSimulator.evolve_fast benchmark()

# Timer unit: 3.95062e-07 s

# Total time: 63.3958 s
# File: C:\Users\user\Documents\python\machinelearning\optimize\simul_optimize.py
# Function: evolve_fast at line 15

# Line #      Hits         Time  Per Hit   % Time  Line Contents
# ==============================================================
#     15                                               def evolve_fast(self, dt):
#     16         1         15.0     15.0      0.0          timestep = 0.00001
#     17         1         18.0     18.0      0.0          nsteps = int(dt/timestep)
#     18                                                   
#     19      1001       4574.0      4.6      0.0          for p in self.particles:
#     20      1000       3492.0      3.5      0.0              t_x_ang = timestep* p.ang_vel
#     21  10001000   26026360.0      2.6     16.2              for i in range(nsteps):
#     22  10000000   58630315.0      5.9     36.5                  norm = (p.x**2+p.y**2)**0.5
#     23  10000000   35248966.0      3.5     22.0                  p.x, p.y = (p.x - t_x_ang*p.y/norm,
#     24  10000000   40556655.0      4.1     25.3                             p.y + t_x_ang*p.x/norm)

    
#加速了約一倍

In [9]:
import dis
from simul import ParticleSimulator
dis.dis(ParticleSimulator.evolve)

 16           0 LOAD_CONST               1 (1e-05)
              2 STORE_FAST               2 (timestep)

 17           4 LOAD_GLOBAL              0 (int)
              6 LOAD_FAST                1 (dt)
              8 LOAD_FAST                2 (timestep)
             10 BINARY_TRUE_DIVIDE
             12 CALL_FUNCTION            1
             14 STORE_FAST               3 (nsteps)

 19          16 SETUP_LOOP             134 (to 152)
             18 LOAD_GLOBAL              1 (range)
             20 LOAD_FAST                3 (nsteps)
             22 CALL_FUNCTION            1
             24 GET_ITER
        >>   26 FOR_ITER               122 (to 150)
             28 STORE_FAST               4 (i)

 20          30 SETUP_LOOP             116 (to 148)
             32 LOAD_FAST                0 (self)
             34 LOAD_ATTR                2 (particles)
             36 GET_ITER
        >>   38 FOR_ITER               106 (to 146)
             40 STORE_FAST               5 (p)

 22    

In [10]:
from simul_optimize import ParticleSimulator
dis.dis(ParticleSimulator.evolve_fast)

 16           0 LOAD_CONST               1 (1e-05)
              2 STORE_FAST               2 (timestep)

 17           4 LOAD_GLOBAL              0 (int)
              6 LOAD_FAST                1 (dt)
              8 LOAD_FAST                2 (timestep)
             10 BINARY_TRUE_DIVIDE
             12 CALL_FUNCTION            1
             14 STORE_FAST               3 (nsteps)

 19          16 SETUP_LOOP             112 (to 130)
             18 LOAD_FAST                0 (self)
             20 LOAD_ATTR                1 (particles)
             22 GET_ITER
        >>   24 FOR_ITER               102 (to 128)
             26 STORE_FAST               4 (p)

 20          28 LOAD_FAST                2 (timestep)
             30 LOAD_FAST                4 (p)
             32 LOAD_ATTR                2 (ang_vel)
             34 BINARY_MULTIPLY
             36 STORE_FAST               5 (t_x_ang)

 21          38 SETUP_LOOP              86 (to 126)
             40 LOAD_GLOBAL           

In [11]:
from simul import benchmark

In [12]:
%load_ext memory_profiler

In [13]:
%mprun -f benchmark benchmark()

# Line #    Mem usage    Increment   Line Contents
# ================================================
#    114     60.2 MiB     60.2 MiB   def benchmark():
#    115     60.3 MiB      0.0 MiB       particles = [Particle(uniform(-1.0, 1.0),
#    116                                                      uniform(-1.0, 1.0),
#    117                                                      uniform(-1.0, 1.0))
#    118     60.3 MiB      0.0 MiB                   for i in range(1000)]
#    119     60.3 MiB      0.0 MiB       simulator = ParticleSimulator(particles)
#    120     60.2 MiB      0.0 MiB       simulator.evolve(0.1)
